# 1 Introduction to competitionm
Now, the goal of the competition is to "build a better music recommendation system" using dataset from KKBOX. More specifically one is to predict the chances of a user listening to song repetitvely within a time period from an event triggered the first listening. 

Now, in the training set we are dealt
* The user id
* Song id
* Where the event of listening to a new song was triggered
* The layout name seen by the user
* Where the song was found
* Finally the target: 1 if the user listened to the same song within the same month and 0 if not. 

Additionally we have the songs dataset that describes the song id and the members dataset containing the user information.  We also have extra information about each song as well. 

Now let us load the modules and datasets.



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
from datetime import datetime
import matplotlib.pyplot as plt
import copy 
import re
from sklearn import preprocessing 
from sklearn.model_selection import GridSearchCV, train_test_split, KFold, StratifiedKFold
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output

In [ ]:
# LOad data ###
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
songs = pd.read_csv('../input/songs.csv')
song_info = pd.read_csv('../input/song_extra_info.csv')
members = pd.read_csv('../input/members.csv')
sample_submission = pd.read_csv('../input/sample_submission.csv')
print("| fin")

# 2.Study of users 
We can begin our study by exploring the training set

In [ ]:
print("Shape of Training set: ",train.shape)
print("Column names: ",train.columns.values)
users = train.loc[:,'msno'].values
unique_users = list(set(users))
print("Number of unique Users: ",len(unique_users))

Hence there are 30,755 users divided over 7,377,418 observation. Talk about big data! Now we can merge on the extra data in order to get a complete dataset.

# 3. Merge in members, songs and song info into the data

We start by merging in the extra data in order to have a complete dataset

In [ ]:
## Merge in data into the training set 

train = pd.merge(train,members,how="left",on="msno")
print("Shape new train: ",train.shape)
train = pd.merge(train,songs,how="left",on="song_id")
print("Shape new train: ",train.shape)
train = pd.merge(train,song_info,how="left",on="song_id")
print("Shape new train: ",train.shape)


In [ ]:
# Merge test data as well 
test = pd.merge(test,members,how="left",on="msno")
print("Shape new test: ",test.shape)
test = pd.merge(test,songs,how="left",on="song_id")
print("Shape new test: ",test.shape)
test = pd.merge(test,song_info,how="left",on="song_id")

print("Shape new test: ",test.shape)

# 4. Study the Missing Values

In [ ]:
## Missing data analysis curtesy https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data_train = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print("| Top Missing data in Training data|")
print(missing_data_train.head(20))

##
total = test.isnull().sum().sort_values(ascending=False)
percent = (test.isnull().sum()/test.isnull().count()).sort_values(ascending=False)
missing_data_test = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print("| Top Missing data in Test data |")
print(missing_data_test.head(20))


Hence we see that there is a 12 variables that have missing values. 3 of these source_screen_name, source_system_tab and source types are found in the training set. We should not remove these columns but rather find values to insert instead of the NaNs. However 

Let us study the columns in order to find replacements for the NaNs.

# 4.1 Study the types of data 

In [ ]:
train_objects = train.columns.to_series().groupby(train.dtypes).groups

kys = list(train_objects.keys())
print(train_objects)
train_objects[kys[0]]
train_cpy = train
#for col in train_cpy.columns.values:
#    print(train_cpy[col].dtype)
#    print("Col: ",col," Number of values: ",train_cpy[col].value_counts().shape[0])
#    print(train_cpy[col].value_counts())

After Inspection we have that the columns
* msno: 30,755 ids (strings)
* song_id: 359,966 song ids (strings)
* source_system_tab: 8 unique objects (strings)
* source_screen_name: 20 unique objects(strings)
* source_type: 12 unique objects(strings)
* target: 2 unique ints
* city: 21 unique ints 
* bd: 92 unique ints
* gender: 2 unqiue values (strings)
* registered_via: 5 unqiue ints
* registration_init_time: 3811 unique ints (dates)
* expiration_date: 1395 unique ints (dates)
* song_length: 50266 unique doubles
* genre_ids: 572 unique strings, but cases of int | int ... | ... int as the music overlaps genres (string)
* artist_name: 40,582 strings
* composer: 76,064 unqque strings
* lyricist: 33,888 unqiue strings
* language: 10 unqiue doubles. One -1 value 
* name: 234,144 unique song names (strings)
* isrc: 269,760 unique strings

Hence we should (1) encode the string objects:
* source_system_tab: 8 unique objects (strings)
* source_screen_name: 20 unique objects(strings)
* source_type: 12 unique objects(strings)
* gender: 2 unqiue values (strings)
* artist_name: 40,582 strings
* composer: 76,064 unqque strings
* lyricist: 33,888 unqiue strings
* name: 234,144 unique song names (strings)

to integer values. Additionally turn:
* registration_init_time: 3811 unique ints (dates)
* expiration_date: 1395 unique ints (dates)
to pd datetimes (2). 

And extract information from the isrc
* isrc: 269,760 unique strings
as the isrc is on the form  "CC-XXX-YY-NNNNN" where CC is the country code, XXX is the 3 character registrant code. YY last two digits of reference year. NNNNN 5 digit identifying the song. 

and seperate the genre ids:
* genre_ids: 572 unique strings, but cases of int | int ... | ... int as the music overlaps genres (string)

 Hence we will end up with many new categorical variables

# 5. Process the Data

# 5.1 Artist_name splitting

It would be interesting to split the artist name if there is featured artists in the artist_name. It might be that a user is a big fan of Future (which is featured on everything) and thus if he appears on a track this might increase the chance of a second listen.

I have only began experimenting with this and unfortunately the workbook does not allow me to run the code to the end. Probably because of inefficient coding, sorry.

In [ ]:
## Experiment with the regular expressions on the artist name

'''
print(re.split("(\w+)?[feat||]*(\w+?[\s\w+])","sia feat kanye|rkelly|lil wayne|beyonce"))
print(re.split("(\w+)?[featuring|feat|feat.||]*(\w+?[\s\w+])","sia featuring kanye|rkelly|lil wayne|beyonce"))
print(re.split("[?\s+]feat?uring[?\s+]|\|","sia feat. kanye | lil wayne"))

fit_label = preprocessing.LabelEncoder().fit(['one','two','three'])
print(fit_label.transform(['one','one']))
'''

We start by encoding the strings objects to ints

In [ ]:
'''
## Study the string objects
train["artist_name"] = train["artist_name"].str.lower()
counts = train["artist_name"].value_counts()
uniq = set(list((counts.index)))
#print(uniq)
#print(re.findall("feat|(s+) ?\|","sia feat kanye, rkelly"))

#Have | and feat seperating artists
where = train["artist_name"].str.find("feat")>0 
print("..")
split_train = train["artist_name"].str.split("[?\s+]feat?uring.[?\s+]|\|",expand=True)
print("..")
split_train = split_train.apply(lambda x: x.str.strip())
uniq_names = list(pd.unique(split_train.values.ravel('K')))
print("..")
la = split_train.apply(lambda x: [uniq_names.index(v) for v in x] ,axis = 1)
print("..")
la.colnames = ['MainArtist'] + ["FeatArtist"+str(c) for c in la.colnames.values[1:]]
print(la)

train[la.colnames] = la
''''''

# 5.2 Code the categorical variables as categorical variables
A no brainer this one. 

In [ ]:
string_objs = ["source_system_tab","source_screen_name",'source_type','gender','artist_name',
              'composer','lyricist','name']
for col in string_objs:
    train[col] = train[col].astype('category').cat.codes
train_objects = train.columns.to_series().groupby(train.dtypes).groups
print(train_objects)

Next step is to turn the datetimes into dates

In [ ]:
date_objs = ['registration_init_time','expiration_date']

train['registration_init_time']=pd.to_datetime(train['registration_init_time'], format='%Y%m%d')
train['expiration_date']=pd.to_datetime(train['expiration_date'], format='%Y%m%d')


# 5.3 Extra Information from isrc code
The isrc code is " International Standard Recording Code (ISRC) is an international standard code for uniquely identifying sound recordings and music video recordings" according to wikipedia. 

You can find an explanation to what the code contains and finally how to split it for every song. 

In [ ]:
isrc = ['isrc']
print("| Example extraction from ISRC |")
print('ISRC:      ',train[isrc].values[0][0])
print('Country:   ',train[isrc].values[0][0][0:2])
print('Reg code:  ',train[isrc].values[0][0][2:5])
print('Year:      ',train[isrc].values[0][0][5:7])
print('Unique id: ',train[isrc].values[0][0][7:])
print("|------------------------------|")
#print(train['isrc'].str.extract('(.{2,2})' * 4))
isrc_cols = ['isrc_c','isrc_rc','isrc_yr','isrc_ud']
train['isrc_c'],train['isrc_rc'],train['isrc_yr'],train['isrc_ud'] = map(train['isrc'].str.slice, [0, 2, 5,7], [2, 5, 7,12])
train[isrc_cols] = train[isrc_cols].fillna(value=-1)

train['isrc_c'] = train['isrc_c'].astype('category').cat.codes
train['isrc_rc'] = train['isrc_rc'].astype('category').cat.codes
train['isrc_yr'] = train['isrc_yr'].astype("int")
train['isrc_ud'] =train['isrc_ud'].astype('category').cat.codes
#print([i[0:2] for i in train[isrc].values])

# 5.4 Split the genre id into several genres and encode them 
This goes back to the "artists featured on a song". The format of the gendre_id data is genre1|genre2|....|genren and could therefore be split into n columns each a integer. So we don't need to categorise the data, neat!

In [ ]:
print(train['genre_ids'].value_counts())
genres_new = train['genre_ids'].str.split('|', expand=True)
genres_new = genres_new.fillna(value=-1)
genres_new.columns = ["genre_"+str(c) for c in genres_new.columns]

train[genres_new.columns] = genres_new.astype("int64")


In [ ]:
train_objects = train.columns.to_series().groupby(train.dtypes).groups
print(train_objects)

So finally we have turned all the dtype('O') objects to categorical variables. Now we don't need to keep genre_ids and isrc so we remove these.

In [ ]:
train = train.drop(['genre_ids','isrc'],axis=1)
print(train.columns.values)

# 6. NaN handeling
This is partly dealt with by setting variables with NaNs to -1. 

# 7.  Feature engineering
Now that we have identified some variables that might help explain whether a user repetitively listenings to the same song within a month. 

The natural next step is to create new feratures based on intuition that mighrt help explain whether a user will listen to a song a second time or not. 

# 7.1 Calculate the time of the subscription

The logic goes that if the user have had a subscription over a longer time this might increase the chances of listening to the same song over again. 

In [ ]:
## Feature Engineering ###
# Days since registration
diff_days = (train['expiration_date']-train['registration_init_time']).dt.days
train['diff_days'] = diff_days

# 7.2 Calculate the number of genres each song belongs to
Maybe some perfer to listen to music that spans various genres or not.

In [ ]:
# Number of Genres 
train['num_genres'] = (train[genres_new.columns]!=-1).sum(axis=1)

# 7.3 Split the songs length into n bins
If we divide into n=3 bins we could have short songs, medium length songs and long songs. Maybe a user finds it easier to listen to a short song a second time than a long one. 

In [ ]:
# Split song length
train['song_length_bins'] = pd.qcut(train['song_length'],10,labels=range(10))

# 7.4 Split the ages into bins
This is done under the assumption that the chance of listening to a song over again is greater in one age group than another. This is can be caused by the intuition that use frequency is greater amongs people in their 20s than in their 60s. But hey it can prove not to be the case!

In [ ]:
# Age bins

## Adjust ages ##
#print(train.loc[train['bd'] >90,'bd'])
print("| Ages less than 16: ", train.loc[train['bd'] <-1,'bd'].shape[0])
print("| Ages greater than 90: ", train.loc[train['bd'] >90,'bd'].shape[0])
train.loc[train['bd'] >90,'bd'] = -1
train.loc[train['bd'] <10,'bd']= -1
#train['bd_age'] = pd.qcut(train['bd'],bins,labels=False)

# 8.0 Random person analysis
Let us now look at one random (lucky) person and see how the original and created variables helps explain the chance of listening to a song twice within a month. 

In [ ]:
var = 'artist_name'
random_user = train['msno'].iloc[0]
random_user_data = train[train['msno']==random_user]
random_user_data = random_user_data.drop(["msno","song_id",'registration_init_time', 'expiration_date'],axis=1)
print(random_user_data.columns.values)
print("| Correlation study |")
print(random_user_data.corrwith(random_user_data['target'],axis=0).sort_values(ascending=False))
tar_0 = random_user_data[random_user_data['target']==0]
tar_1 = random_user_data[random_user_data['target']==1]
val_0_var =tar_0[var].value_counts()
val_0_var = val_0_var/val_0_var.sum(axis=0)
val_1_var = tar_1[var].value_counts()
val_1_var = val_1_var/val_1_var.sum(axis=0)
comp = pd.concat([val_0_var, val_1_var],axis=1,names=['0','1'])
comp.columns = ['target 0','target 1']
comp['absdiff'] = abs(comp['target 0'] - comp['target 1'])
print()
print("| ",var," vs. target |")
print(comp.sort_values(by="absdiff",ascending=False))
#plt.scatter(random_user_data[var],random_user_data['target'])
print(train[var].value_counts())

# 8. Complete dataset
Finally we have a complete dataset which is ready to be trained and tested. As one might observe I have not carried out the feature engineering for the test set. This should however be straight forward: just change train to train + test in the beginning and your off. 




# 9 Modelling of the data
This is not currently finished. 

In [ ]:
'''
## Run V fold CV for XGBOOST ##
kfold = 5
skf = StratifiedKFold(n_splits=kfold, random_state=42)
for i, (train_index, test_index) in enumerate(skf.split(X_data, y_data)):
    print("fold ",i," of ",kfold)
    X_train, X_valid = X_data[train_index, :], X_data[test_index, :]
    y_train, y_valid = y_data[train_index],    y_data[test_index]
    
    
    d_train = xgb.DMatrix(X_train, y_train,missing=-999.0)
    d_valid = xgb.DMatrix(X_valid, y_valid,missing=-999.0)
    #d_test = xgb.DMatrix(test_data)

    watchlist = [(d_train, 'train'), (d_valid, 'valid')]

    # Model XGBOOST
    print(" - model")
    xgb_mdl = xgb.train(xgb_params, d_train,100, watchlist, early_stopping_rounds=100, feval='auc', maximize=True,
                        verbose_eval=1)
    print(" - predict")
    xgb_predict = xgb_mdl.predict(xgb.DMatrix(X_valid),ntree_limit=xgb_mdl.best_ntree_limit)
'''